# Control The Robot with Joystick

In [ ]:
from FR3Py.interfaces import FR3Real
robot = FR3Real()

Test the interface by reading the robot joints:

In [ ]:
robot.get_state()

In [ ]:
from pyJoy.joy_daq import JoyManager
import time    
joy1  = JoyManager()
joy1.start_daq(joy_idx=0)
joy1.offset_calibration()
time.sleep(3)

In [ ]:
from FR3Py.cameras import RealSenseCamera
camera = RealSenseCamera(VGA=False, enable_imu=False, enable_ir=False)

In [ ]:
import time
import numpy as np
from FR3Py.controllers.waypoint_controller import WaypointController
import datetime
import os
import cv2

current_datetime = datetime.datetime.now()
formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
shutter_old = 0 
dataset_path = 'dataset/scene2'

def write_depth(depth, filename):
    depth = depth.astype(np.uint16)
    cv2.imwrite(filename, depth)

def write_color(color, filename):
    color = color.astype(np.uint8)
    cv2.imwrite(filename, color)

def write_pose(pose, filename):
    np.savetxt(filename, pose)
    

controller = WaypointController(kp=4)
# Read the initila pose of the robot
time.sleep(1)
state = robot.get_state()

q, dq = state['q'], state['dq']
p0 = controller.robot.getInfo(q,dq)['P_EE']
R0 = controller.robot.getInfo(q,dq)['R_EE']

T0 = np.vstack([np.hstack([R0, p0.reshape(3,1)]), np.array([0,0,0,1])])
start_time = time.time()

x0, y0, z0 = 0.0, 0.0, 0.0 
R0 = np.eye(3)
ef_poses = []
stamps = []
while True:
    trans = controller.robot.getInfo(q,dq)['P_EE']
    rot = controller.robot.getInfo(q,dq)['R_EE']
    stamps.append(time.time())
    pose = np.vstack([np.hstack([rot, trans.reshape(3,1)]), np.array([0,0,0,1])])
    ef_poses.append(pose)
    
    analog, digital = joy1.read_values()
    cmd = np.array([analog[0],analog[1],analog[2]])
    for i in range(3):
        if np.abs(cmd[i])<0.1:
            cmd[i] = 0 

    if digital[0]==0:
        y0 = y0 - cmd[0]*0.2/100
        x0 = x0 - cmd[1]*0.2/100
        z0 = z0 + cmd[2]*0.2/100
    else:
        omega_hat = np.array([[ 0,       -cmd[2],  cmd[1]],
                              [ cmd[2],   0,      -cmd[0]],
                              [-cmd[1],   cmd[0],      0]])
        R0 = R0@(np.eye(3)+omega_hat/100)
    
    camera.grab_frames()
    cv2.imshow('color', camera.color_frame)
    cv2.waitKey(1)
    # Take a snapshot if required
    shutter = digital[2]
    if shutter==1 and shutter_old==0:
        stamp_ns = int(time.time()*1e9)
        shutter_old = shutter
        depth = camera.depth_frame
        color = camera.color_frame
        print('Taking a picture')
        write_depth(depth, os.path.join(dataset_path, 'depth', f'{stamp_ns}.png'))
        write_color(color, os.path.join(dataset_path, 'color', f'{stamp_ns}.png'))
        write_pose(pose, os.path.join(dataset_path, 'pose', f'{stamp_ns}.txt'))
    else:
        shutter_old = shutter
    # time.sleep(0.01)
    T= T0@np.vstack([np.hstack([R0, np.array([x0,y0,z0]).reshape(3,1)]), np.array([0,0,0,1])])
    state = robot.get_state()
    q, dq = state['q'], state['dq']
    cmd = controller.compute(q,dq, T_cmd=T)
    robot.send_joint_command(cmd[0:7])